# Feature Creation & Final Cleanup On Unnecessary Columns & Label Encoding Categorical Variables

In [102]:
import pandas as pd
import numpy as np
import seaborn as sns

### Initial Discussion Around Dummies

Dumify condition, cylinders, fuel, title-status, transmission, drive, type, state, car_age  = (posting date - year) and drop year and posting date, vintage (yes or no) 

Create features
car_age = posting year - production year 
is_vintage = greater than 50 car_age
is_neutral = colors like (white, black, silver, gray) vs 

In [103]:
dfcar3 = pd.read_csv("/Volumes/GoogleDrive/My Drive/MScA 2022 WINTER/MSCA 31008 5 Data Mining Principles/Project/dfcar2.csv")
len(dfcar3)

183783

In [104]:
dfcar3.head(2)

,Unnamed: 0,price,year,condition,cylinders,fuel,odometer,title_status,transmission,drive,...,match_score,manufacturer_msrp,model_msrp,year_msrp,cylinders_msrp,vstyle_msrp,MSRP,wd_msrp,trans_msrp,quarter
0,1,22590,2010.0,good,8.0,gas,71229.0,clean,other,4wd,...,1.0,Chevrolet,Silverado 1500,2015.0,8.0,Crew Cab Pickup,46110.0,4WD,AUTOMATIC,Q2
1,7,35000,2019.0,excellent,6.0,gas,43000.0,clean,automatic,4wd,...,2.0,Toyota,Tacoma,2015.0,6.0,Crew Cab Pickup,27355.0,4WD,MANUAL,Q2


In [105]:
dfcar3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183783 entries, 0 to 183782
Data columns (total 26 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         183783 non-null  int64  
 1   price              183783 non-null  int64  
 2   year               183783 non-null  float64
 3   condition          183783 non-null  object 
 4   cylinders          183783 non-null  float64
 5   fuel               183783 non-null  object 
 6   odometer           183783 non-null  float64
 7   title_status       183783 non-null  object 
 8   transmission       183783 non-null  object 
 9   drive              183783 non-null  object 
 10  type               183783 non-null  object 
 11  paint_color        128366 non-null  object 
 12  description        183782 non-null  object 
 13  state              183783 non-null  object 
 14  posting_date       183783 non-null  object 
 15  matched_model      183783 non-null  object 
 16  ma

### Change Quarter To Numeric Value

In [106]:
float_t2 = dfcar3.select_dtypes(include=['float'])
object_t2 = dfcar3.select_dtypes(include=['object'])

In [107]:
object_t2.head(2)

,condition,fuel,title_status,transmission,drive,type,paint_color,description,state,posting_date,matched_model,manufacturer_msrp,model_msrp,vstyle_msrp,wd_msrp,trans_msrp,quarter
0,good,gas,clean,other,4wd,pickup,blue,Carvana is the safer way to buy a car During t...,al,2021-05-04 17:31:08+00:00,Chevrolet Silverado 1500,Chevrolet,Silverado 1500,Crew Cab Pickup,4WD,AUTOMATIC,Q2
1,excellent,gas,clean,automatic,4wd,truck,grey,Selling my 2019 Toyota Tacoma TRD Off Road Dou...,al,2021-05-03 17:12:59+00:00,Toyota Tacoma,Toyota,Tacoma,Crew Cab Pickup,4WD,MANUAL,Q2


In [108]:
dfcar3['quarter'] = dfcar3['quarter'].str.lstrip("Q").astype("int")

In [109]:
dfcar3 = dfcar3.drop("Unnamed: 0", axis=1)

### Car_age & is_vintage features are created

In [120]:
# add year_posted
dfcar3['year_posted']=pd.DatetimeIndex(dfcar3['posting_date']).year

In [121]:
# add car_age
dfcar3['car_age']=dfcar3['year_posted']-dfcar3['year']

In [122]:
# add dummy: vintage yes/no
dfcar3['is_vintage'] = dfcar3['car_age'].apply(lambda x: 1 if x >50 else 0)

In [123]:
dfcar3['is_vintage'].value_counts()

0    182882
1       901
Name: is_vintage, dtype: int64

In [124]:
dfcar3['car_age'].value_counts()

8.0      14629
4.0      14564
6.0      14470
7.0      14348
3.0      13162
         ...  
89.0         1
121.0        1
83.0         1
70.0         1
116.0        1
Name: car_age, Length: 83, dtype: int64

### Paint_color (neutral vs colorful)

In [125]:
dfcar3['paint_color'].value_counts()

white     29430
black     27290
silver    20414
blue      13883
grey      13238
red       12394
custom     4014
green      3024
brown      2802
orange      773
yellow      747
purple      357
Name: paint_color, dtype: int64

In [126]:
dfcar3['is_color_neutral'] = dfcar3['paint_color'].apply(lambda x: 1 if x in ['white', 'black', 'silver', 'grey'] else 0)

In [127]:
dfcar3['is_color_neutral'].value_counts()

0    93411
1    90372
Name: is_color_neutral, dtype: int64

### Drop Unnecessary Columns 

In [128]:
dfcar4 = dfcar3

In [129]:
dfcar4 = dfcar4.drop(['matched_model', 'match_score',
       'manufacturer_msrp', 'model_msrp', 'year_msrp', 'cylinders_msrp',
       'vstyle_msrp','wd_msrp', 'trans_msrp','year_posted', 'posting_date','paint_color', 'description'], axis=1)

In [130]:
dfcar4.dtypes

price                 int64
year                float64
condition            object
cylinders           float64
fuel                 object
odometer            float64
title_status         object
transmission         object
drive                object
type                 object
state                object
MSRP                float64
quarter               int64
car_age             float64
is_vintage            int64
is_color_neutral      int64
dtype: object

In [131]:
null = dfcar4.isnull().sum()
print(null[null>0])

MSRP    6369
dtype: int64


In [132]:
dfcar5 = dfcar4.dropna(subset=["MSRP"])

In [133]:
null = dfcar5.isnull().sum()
print(null[null>0])

Series([], dtype: int64)


### Using LabelEncoder, turn categorical variables to numerical variables

In [147]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
dfcar7 = dfcar5
dfcar7[['condition', 'cylinders', 'fuel', 'title_status', 'transmission',
       'drive', 'type', 'state']]=dfcar7[['condition', 'cylinders', 'fuel', 'title_status', 'transmission',
       'drive', 'type', 'state']].apply(le.fit_transform)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [148]:
dfcar7.head(2)

,price,year,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,state,MSRP,quarter,car_age,is_vintage,is_color_neutral
0,22590,2010.0,2,4,2,71229.0,0,2,0,8,1,46110.0,2,11.0,0,0
1,35000,2019.0,0,3,2,43000.0,0,0,0,10,1,27355.0,2,2.0,0,1


In [149]:
dfcar7.value_counts()

price       year    condition  cylinders  fuel  odometer  title_status  transmission  drive  type  state  MSRP     quarter  car_age  is_vintage  is_color_neutral
500         2013.0  1          1          2     130000.0  5             0             0      9     4      22560.0  2        8.0      0           0                   77
100         2011.0  0          3          2     100000.0  0             0             0      0     35     23995.0  2        10.0     0           0                   53
25998       2019.0  0          1          2     27056.0   0             0             1      9     26     26150.0  2        2.0      0           0                   31
1           2016.0  0          3          2     57722.0   0             2             0      12    9      27895.0  2        5.0      0           1                   31
388         2002.0  1          1          2     150000.0  0             0             1      9     6      19450.0  2        19.0     0           0                   3

In [150]:
dfcar7.to_csv("labelencoder2.csv", index=False)